## Spark SQL

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
import random

#### RDD VS Data Frame

In [0]:
people_data = [("Brooke", 20), ("Denny", 31), ("Jules", 30),
("TD", 35), ("Brooke", 25), ("TD", 60), ("Brooke", 30)]

##### RDD

In [0]:
dataRDD = sc.parallelize(people_data)

agesRDD = (dataRDD.map(lambda x: (x[0], (x[1], 1))) 
           .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) 
           .map(lambda x: (x[0], x[1][0]/x[1][1])))
agesRDD.collect()

Out[264]: [('Brooke', 25.0), ('Denny', 31.0), ('TD', 47.5), ('Jules', 30.0)]

##### DataFrame

In [0]:
data_df = spark.createDataFrame(people_data, ["name", "age"])
avg_df = data_df.groupBy("name").agg(avg("age"))
avg_df.show()

+------+--------+
 name|avg(age)|
+------+--------+
Brooke| 25.0|
 Denny| 31.0|
 Jules| 30.0|
 TD| 47.5|
+------+--------+

### Creating DataFrame

#### From RDD

In [0]:
some_rdd = sc.parallelize(range(1,10)).map(lambda x : (x, random.randint(0,100)))

In [0]:
some_rdd.collect()

Out[183]: [(1, 62),
 (2, 62),
 (3, 12),
 (4, 64),
 (5, 82),
 (6, 44),
 (7, 19),
 (8, 5),
 (9, 48)]

In [0]:
kv_DF = some_rdd.toDF(["key", "value"])
kv_DF.printSchema()

root
-- key: long (nullable = true)
-- value: long (nullable = true)

In [0]:
kv_DF.show()

+---+-----+
key|value|
+---+-----+
 1| 1|
 2| 6|
 3| 36|
 4| 12|
 5| 55|
 6| 12|
 7| 28|
 8| 12|
 9| 61|
+---+-----+

#### From collection

In [0]:
data = [[1, "Jules", "Damji", "https://tinyurl.1", "1/4/2016", 4535, ["twitter", "LinkedIn"]],
           [2, "Brooke","Wenig", "https://tinyurl.2", "5/5/2018", 8908, ["twitter",
    "LinkedIn"]],
           [3, "Denny", "Lee", "https://tinyurl.3", "6/7/2019", 7659, ["web",
    "twitter", "FB", "LinkedIn"]],
           [4, "Tathagata", "Das", "https://tinyurl.4", "5/12/2018", 10568,
    ["twitter", "FB"]],
           [5, "Matei","Zaharia", "https://tinyurl.5", "5/14/2014", 40578, ["web",
    "twitter", "FB", "LinkedIn"]],
           [6, "Reynold", "Xin", "https://tinyurl.6", "3/2/2015", 25568,
    ["twitter", "LinkedIn"]]
          ]



#### Defining a schema

##### Schema Created Programmatically

In [0]:

schema = StructType([
  StructField("Id", IntegerType(), False),
  StructField("First", StringType(), False),
  StructField("Last", StringType(), False),
  StructField("Url", StringType(), True),
  StructField("Published", StringType(), False),
  StructField("Hits", IntegerType(), False),
  StructField("Campaigns", ArrayType(StringType(), True), False)
])

##### Spark Data Types
* BooleanType
* ByteType
* IntegerType
* LongType
* FloatType
* DoubleType
* DecimalType
* StringType
* BinaryType
* TimestampType
* DateType
* ArrayType
* MapType
* StructType
* StructField

In [0]:
blogs_df = spark.createDataFrame(data, schema)
blogs_df.show()
blogs_df.printSchema()

+---+---------+-------+-----------------+---------+-----+--------------------+
 Id| First| Last| Url|Published| Hits| Campaigns|
+---+---------+-------+-----------------+---------+-----+--------------------+
 1| Jules| Damji|https://tinyurl.1| 1/4/2016| 4535| [twitter, LinkedIn]|
 2| Brooke| Wenig|https://tinyurl.2| 5/5/2018| 8908| [twitter, LinkedIn]|
 3| Denny| Lee|https://tinyurl.3| 6/7/2019| 7659|[web, twitter, FB...|
 4|Tathagata| Das|https://tinyurl.4|5/12/2018|10568| [twitter, FB]|
 5| Matei|Zaharia|https://tinyurl.5|5/14/2014|40578|[web, twitter, FB...|
 6| Reynold| Xin|https://tinyurl.6| 3/2/2015|25568| [twitter, LinkedIn]|
+---+---------+-------+-----------------+---------+-----+--------------------+

root
-- Id: integer (nullable = false)
-- First: string (nullable = false)
-- Last: string (nullable = false)
-- Url: string (nullable = true)
-- Published: string (nullable = false)
-- Hits: integer (nullable = false)
-- Campaigns: array (nullable = false)
 |-- element: string (containsNull = true)

In [0]:
blogs_df.columns

Out[189]: ['Id', 'First', 'Last', 'Url', 'Published', 'Hits', 'Campaigns']

In [0]:
blogs_df.schema

Out[190]: StructType(List(StructField(Id,IntegerType,false),StructField(First,StringType,false),StructField(Last,StringType,false),StructField(Url,StringType,true),StructField(Published,StringType,false),StructField(Hits,IntegerType,false),StructField(Campaigns,ArrayType(StringType,true),false)))

#### Creating DataFranes from Data Sources

* text
* json
* csv
* jdbc
* orc
* parquet

`DataFrameReader`

##### Create a DataFrame from text file

In [0]:
%fs head /databricks-datasets/README.md

Databricks Hosted Datasets
==========================

The data contained within this directory is hosted for users to build 
data pipelines using Apache Spark and Databricks.

License
-------
Unless otherwise noted (e.g. within the README for a given data set), the data 
is licensed under Creative Commons Attribution 4.0 International (CC BY 4.0),
which can be viewed at the following url:
[http://creativecommons.org/licenses/by/4.0/legalcode](http://creativecommons.org/licenses/by/4.0/legalcode)

Contributions and Requests
--------------------------
To request or contribute new datasets to this repository, please send an email
to: hosted-datasets@databricks.com.

When making the request, include the README.md file you want to publish. Make
sure the file includes information about the source of the data, the license, 
and how to get additional information. Please ensure the license for this 
data allows it to be hosted by Databricks and consumed by the public.

In [0]:
text_file = spark.read.text("dbfs:/databricks-datasets/README.md")
text_file.show(5, truncate=False)

+----------------------------------------------------------------------+
value |
+----------------------------------------------------------------------+
Databricks Hosted Datasets |
========================== |
 |
The data contained within this directory is hosted for users to build |
data pipelines using Apache Spark and Databricks. |
+----------------------------------------------------------------------+
only showing top 5 rows

##### Create a DataFrame from the NYT COVID data CSV file

In [0]:
%fs head /databricks-datasets/COVID/covid-19-data/us-counties.csv

[Truncated to first 65536 bytes]
date,county,state,fips,cases,deaths
2020-01-21,Snohomish,Washington,53061,1,0
2020-01-22,Snohomish,Washington,53061,1,0
2020-01-23,Snohomish,Washington,53061,1,0
2020-01-24,Cook,Illinois,17031,1,0
2020-01-24,Snohomish,Washington,53061,1,0
2020-01-25,Orange,California,06059,1,0
2020-01-25,Cook,Illinois,17031,1,0
2020-01-25,Snohomish,Washington,53061,1,0
2020-01-26,Maricopa,Arizona,04013,1,0
2020-01-26,Los Angeles,California,06037,1,0
2020-01-26,Orange,California,06059,1,0
2020-01-26,Cook,Illinois,17031,1,0
2020-01-26,Snohomish,Washington,53061,1,0
2020-01-27,Maricopa,Arizona,04013,1,0
2020-01-27,Los Angeles,California,06037,1,0
2020-01-27,Orange,California,06059,1,0
2020-01-27,Cook,Illinois,17031,1,0
2020-01-27,Snohomish,Washington,53061,1,0
2020-01-28,Maricopa,Arizona,04013,1,0
2020-01-28,Los Angeles,California,06037,1,0
2020-01-28,Orange,California,06059,1,0
2020-01-28,Cook,Illinois,17031,1,0
2020-01-28,Snohomish,Washington,53061,1,0
2020-01-29,Maricopa,Arizona,04013,1,0
2020-01-29,Los Angeles,California,06037,1,0
2020-01-29,Orange,California,06059,1,0
2020-01-29,Cook,Illinois,17031,1,0
2020-01-29,Snohomish,Washington,53061,1,0
2020-01-30,Maricopa,Arizona,04013,1,0
2020-01-30,Los Angeles,California,06037,1,0
2020-01-30,Orange,California,06059,1,0
2020-01-30,Cook,Illinois,17031,2,0
2020-01-30,Snohomish,Washington,53061,1,0
2020-01-31,Maricopa,Arizona,04013,1,0
2020-01-31,Los Angeles,California,06037,1,0
2020-01-31,Orange,California,06059,1,0
2020-01-31,Santa Clara,California,06085,1,0
2020-01-31,Cook,Illinois,17031,2,0
2020-01-31,Snohomish,Washington,53061,1,0
2020-02-01,Maricopa,Arizona,04013,1,0
2020-02-01,Los Angeles,California,06037,1,0
2020-02-01,Orange,California,06059,1,0
2020-02-01,Santa Clara,California,06085,1,0
2020-02-01,Cook,Illinois,17031,2,0
2020-02-01,Suffolk,Massachusetts,25025,1,0
2020-02-01,Snohomish,Washington,53061,1,0
2020-02-02,Maricopa,Arizona,04013,1,0
2020-02-02,Los Angeles,California,06037,1,0
2020-02-02,Orange,California,06059,1,0
2020-02-02,San Francisco,California,06075,2,0
2020-02-02,Santa Clara,California,06085,2,0
2020-02-02,Cook,Illinois,17031,2,0
2020-02-02,Suffolk,Massachusetts,25025,1,0
2020-02-02,Snohomish,Washington,53061,1,0
2020-02-03,Maricopa,Arizona,04013,1,0
2020-02-03,Los Angeles,California,06037,1,0
2020-02-03,Orange,California,06059,1,0
2020-02-03,San Francisco,California,06075,2,0
2020-02-03,Santa Clara,California,06085,2,0
2020-02-03,Cook,Illinois,17031,2,0
2020-02-03,Suffolk,Massachusetts,25025,1,0
2020-02-03,Snohomish,Washington,53061,1,0
2020-02-04,Maricopa,Arizona,04013,1,0
2020-02-04,Los Angeles,California,06037,1,0
2020-02-04,Orange,California,06059,1,0
2020-02-04,San Francisco,California,06075,2,0
2020-02-04,Santa Clara,California,06085,2,0
2020-02-04,Cook,Illinois,17031,2,0
2020-02-04,Suffolk,Massachusetts,25025,1,0
2020-02-04,Snohomish,Washington,53061,1,0
2020-02-05,Maricopa,Arizona,04013,1,0
2020-02-05,Los Angeles,California,06037,1,0
2020-02-05,Orange,California,06059,1,0
2020-02-05,San Francisco,California,06075,2,0
2020-02-05,Santa Clara,California,06085,2,0
2020-02-05,Cook,Illinois,17031,2,0
2020-02-05,Suffolk,Massachusetts,25025,1,0
2020-02-05,Snohomish,Washington,53061,1,0
2020-02-05,Dane,Wisconsin,55025,1,0
2020-02-06,Maricopa,Arizona,04013,1,0
2020-02-06,Los Angeles,California,06037,1,0
2020-02-06,Orange,California,06059,1,0
2020-02-06,San Francisco,California,06075,2,0
2020-02-06,Santa Clara,California,06085,2,0
2020-02-06,Cook,Illinois,17031,2,0
2020-02-06,Suffolk,Massachusetts,25025,1,0
2020-02-06,Snohomish,Washington,53061,1,0
2020-02-06,Dane,Wisconsin,55025,1,0
2020-02-07,Maricopa,Arizona,04013,1,0
2020-02-07,Los Angeles,California,06037,1,0
2020-02-07,Orange,California,06059,1,0
2020-02-07,San Francisco,California,06075,2,0
2020-02-07,Santa Clara,California,06085,2,0
2020-02-07,Cook,Illinois,17031,2,0
2020-02-07,Suffolk,Massachusetts,25025,1,0
2020-02-07,Snohomish,Washington,53061,1,0
2020-02-07,Dane,Wisconsin,55025,1,0
2

In [0]:
cov_df = spark.read.csv("/databricks-datasets/COVID/covid-19-data/us-counties.csv")
cov_df.show(5)
cov_df.printSchema()

+----------+---------+----------+-----+-----+------+
 _c0| _c1| _c2| _c3| _c4| _c5|
+----------+---------+----------+-----+-----+------+
 date| county| state| fips|cases|deaths|
2020-01-21|Snohomish|Washington|53061| 1| 0|
2020-01-22|Snohomish|Washington|53061| 1| 0|
2020-01-23|Snohomish|Washington|53061| 1| 0|
2020-01-24| Cook| Illinois|17031| 1| 0|
+----------+---------+----------+-----+-----+------+
only showing top 5 rows

root
-- _c0: string (nullable = true)
-- _c1: string (nullable = true)
-- _c2: string (nullable = true)
-- _c3: string (nullable = true)
-- _c4: string (nullable = true)
-- _c5: string (nullable = true)

In [0]:
cov_df_ = spark.read.csv("/databricks-datasets/COVID/covid-19-data/us-counties.csv", header=True, inferSchema=True)
cov_df_.show()

+----------+-----------+----------+-----+-----+------+
 date| county| state| fips|cases|deaths|
+----------+-----------+----------+-----+-----+------+
2020-01-21| Snohomish|Washington|53061| 1| 0|
2020-01-22| Snohomish|Washington|53061| 1| 0|
2020-01-23| Snohomish|Washington|53061| 1| 0|
2020-01-24| Cook| Illinois|17031| 1| 0|
2020-01-24| Snohomish|Washington|53061| 1| 0|
2020-01-25| Orange|California| 6059| 1| 0|
2020-01-25| Cook| Illinois|17031| 1| 0|
2020-01-25| Snohomish|Washington|53061| 1| 0|
2020-01-26| Maricopa| Arizona| 4013| 1| 0|
2020-01-26|Los Angeles|California| 6037| 1| 0|
2020-01-26| Orange|California| 6059| 1| 0|
2020-01-26| Cook| Illinois|17031| 1| 0|
2020-01-26| Snohomish|Washington|53061| 1| 0|
2020-01-27| Maricopa| Arizona| 4013| 1| 0|
2020-01-27|Los Angeles|California| 6037| 1| 0|
2020-01-27| Orange|California| 6059| 1| 0|
2020-01-27| Cook| Illinois|17031| 1| 0|
2020-01-27| Snohomish|Washington|53061| 1| 0|
2020-01-28| Maricopa| Arizona| 4013| 1| 0|
2020-01-28|Los Angeles|California| 6037| 1| 0|
+----------+-----------+----------+-----+-----+------+
only showing top 20 rows

In [0]:
cov_df_.printSchema()

root
-- date: string (nullable = true)
-- county: string (nullable = true)
-- state: string (nullable = true)
-- fips: integer (nullable = true)
-- cases: integer (nullable = true)
-- deaths: integer (nullable = true)

In [0]:
cov_schema = StructType([
  StructField("Date",DateType(),True),
  StructField("county",StringType(),True),
  StructField("state",StringType(),True),
  StructField("fips",IntegerType(),True),
  StructField("cases",IntegerType(),True),
  StructField("deaths",IntegerType(),True)
])

In [0]:
cov_df = spark.read.csv("/databricks-datasets/COVID/covid-19-data/us-counties.csv", header=True, schema=cov_schema)
cov_df.show()

+----------+-----------+----------+-----+-----+------+
 Date| county| state| fips|cases|deaths|
+----------+-----------+----------+-----+-----+------+
2020-01-21| Snohomish|Washington|53061| 1| 0|
2020-01-22| Snohomish|Washington|53061| 1| 0|
2020-01-23| Snohomish|Washington|53061| 1| 0|
2020-01-24| Cook| Illinois|17031| 1| 0|
2020-01-24| Snohomish|Washington|53061| 1| 0|
2020-01-25| Orange|California| 6059| 1| 0|
2020-01-25| Cook| Illinois|17031| 1| 0|
2020-01-25| Snohomish|Washington|53061| 1| 0|
2020-01-26| Maricopa| Arizona| 4013| 1| 0|
2020-01-26|Los Angeles|California| 6037| 1| 0|
2020-01-26| Orange|California| 6059| 1| 0|
2020-01-26| Cook| Illinois|17031| 1| 0|
2020-01-26| Snohomish|Washington|53061| 1| 0|
2020-01-27| Maricopa| Arizona| 4013| 1| 0|
2020-01-27|Los Angeles|California| 6037| 1| 0|
2020-01-27| Orange|California| 6059| 1| 0|
2020-01-27| Cook| Illinois|17031| 1| 0|
2020-01-27| Snohomish|Washington|53061| 1| 0|
2020-01-28| Maricopa| Arizona| 4013| 1| 0|
2020-01-28|Los Angeles|California| 6037| 1| 0|
+----------+-----------+----------+-----+-----+------+
only showing top 20 rows

In [0]:
cov_df.printSchema()

root
-- _c0: string (nullable = true)
-- _c1: string (nullable = true)
-- _c2: string (nullable = true)
-- _c3: string (nullable = true)
-- _c4: string (nullable = true)
-- _c5: string (nullable = true)

##### Create a DataFrame from JSON file

In [0]:
%fs head /databricks-datasets/samples/people/people.json

{"name":"Jane","age":40}
{"name":"Andy","age":30}
{"name":"Justin","age":50}

In [0]:
people_df = spark.read.json("dbfs:/databricks-datasets/samples/people/people.json")
people_df.show()

+---+------+
age|name |
+---+------+
40 |Jane |
30 |Andy |
50 |Justin|
+---+------+

### DataFrames Operations

#### Transformations

* select
* selectExpr
* filter 
* where
* distinct
* dropDuplicates
* sort
* orderBy
* limit
* union
* withColumn
* withColumnRenamed
* drop
* sample
* join
* groupBy
* describe

#### Action

* show([numRows], [[truncate]])
* head([n])
* first([n])
* take(n)
* takeAsList(n)
* collect()
* count()

### Working with DataFrames

In [0]:
cov_df.show()

+----------+-----------+----------+-----+-----+------+
 Date| county| state| fips|cases|deaths|
+----------+-----------+----------+-----+-----+------+
2020-01-21| Snohomish|Washington|53061| 1| 0|
2020-01-22| Snohomish|Washington|53061| 1| 0|
2020-01-23| Snohomish|Washington|53061| 1| 0|
2020-01-24| Cook| Illinois|17031| 1| 0|
2020-01-24| Snohomish|Washington|53061| 1| 0|
2020-01-25| Orange|California| 6059| 1| 0|
2020-01-25| Cook| Illinois|17031| 1| 0|
2020-01-25| Snohomish|Washington|53061| 1| 0|
2020-01-26| Maricopa| Arizona| 4013| 1| 0|
2020-01-26|Los Angeles|California| 6037| 1| 0|
2020-01-26| Orange|California| 6059| 1| 0|
2020-01-26| Cook| Illinois|17031| 1| 0|
2020-01-26| Snohomish|Washington|53061| 1| 0|
2020-01-27| Maricopa| Arizona| 4013| 1| 0|
2020-01-27|Los Angeles|California| 6037| 1| 0|
2020-01-27| Orange|California| 6059| 1| 0|
2020-01-27| Cook| Illinois|17031| 1| 0|
2020-01-27| Snohomish|Washington|53061| 1| 0|
2020-01-28| Maricopa| Arizona| 4013| 1| 0|
2020-01-28|Los Angeles|California| 6037| 1| 0|
+----------+-----------+----------+-----+-----+------+
only showing top 20 rows

#### Load data

In [0]:
%fs ls /databricks-datasets/COVID/CSSEGISandData/csse_covid_19_data/csse_covid_19_daily_reports/

path,name,size
dbfs:/databricks-datasets/COVID/CSSEGISandData/csse_covid_19_data/csse_covid_19_daily_reports/.gitignore,.gitignore,9
dbfs:/databricks-datasets/COVID/CSSEGISandData/csse_covid_19_data/csse_covid_19_daily_reports/01-01-2021.csv,01-01-2021.csv,566153
dbfs:/databricks-datasets/COVID/CSSEGISandData/csse_covid_19_data/csse_covid_19_daily_reports/01-02-2021.csv,01-02-2021.csv,566475
dbfs:/databricks-datasets/COVID/CSSEGISandData/csse_covid_19_data/csse_covid_19_daily_reports/01-03-2021.csv,01-03-2021.csv,566537
dbfs:/databricks-datasets/COVID/CSSEGISandData/csse_covid_19_data/csse_covid_19_daily_reports/01-04-2021.csv,01-04-2021.csv,566578
dbfs:/databricks-datasets/COVID/CSSEGISandData/csse_covid_19_data/csse_covid_19_daily_reports/01-05-2021.csv,01-05-2021.csv,566788
dbfs:/databricks-datasets/COVID/CSSEGISandData/csse_covid_19_data/csse_covid_19_daily_reports/01-06-2021.csv,01-06-2021.csv,566804
dbfs:/databricks-datasets/COVID/CSSEGISandData/csse_covid_19_data/csse_covid_19_daily_reports/01-07-2021.csv,01-07-2021.csv,566852
dbfs:/databricks-datasets/COVID/CSSEGISandData/csse_covid_19_data/csse_covid_19_daily_reports/01-08-2021.csv,01-08-2021.csv,567013
dbfs:/databricks-datasets/COVID/CSSEGISandData/csse_covid_19_data/csse_covid_19_daily_reports/01-09-2021.csv,01-09-2021.csv,566974


In [0]:
%fs head /databricks-datasets/COVID/CSSEGISandData/csse_covid_19_data/csse_covid_19_daily_reports/01-31-2021.csv


[Truncated to first 65536 bytes]
FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
,,,Afghanistan,2021-01-26 05:22:02,33.93911,67.709953,54672,2385,47365,4922,Afghanistan,140.44266617989192,4.362379280070237
,,,Albania,2021-01-26 05:22:02,41.1533,20.1683,72812,1324,44291,27197,Albania,2530.127180485093,1.8183815854529473
,,,Algeria,2021-01-26 05:22:02,28.0339,1.6596,105854,2866,72143,30845,Algeria,241.39448632954975,2.70750278685737
,,,Andorra,2021-01-26 05:22:02,42.5063,1.5218,9596,97,8724,775,Andorra,12419.594900666538,1.0108378491037933
,,,Angola,2021-01-26 05:22:02,-11.2027,17.8739,19476,461,17327,1688,Angola,59.25832528353995,2.3670158143355926
,,,Antigua and Barbuda,2021-01-26 05:22:02,17.0608,-61.7964,201,6,170,25,Antigua and Barbuda,205.25283882035785,2.985074626865672
,,,Argentina,2021-01-26 05:22:02,-38.4161,-63.6167,1874801,47034,1666527,161240,Argentina,4148.1773839179705,2.5087462616032314
,,,Armenia,2021-01-26 05:22:02,40.0691,45.0382,166094,3047,154932,8115,Armenia,5605.159768010221,1.834503353522704
,,Australian Capital Territory,Australia,2021-01-26 05:22:02,-35.4735,149.0124,118,3,115,0,"Australian Capital Territory, Australia",27.563653352020555,2.542372881355932
,,New South Wales,Australia,2021-01-26 05:22:02,-33.8688,151.2093,5091,54,3242,1795,"New South Wales, Australia",62.71249076127125,1.060695344725987
,,Northern Territory,Australia,2021-01-26 05:22:02,-12.4634,130.8456,98,0,90,8,"Northern Territory, Australia",39.90228013029316,0.0
,,Queensland,Australia,2021-01-26 05:22:02,-27.4698,153.0251,1305,6,1278,21,"Queensland, Australia",25.51070276610302,0.45977011494252873
,,South Australia,Australia,2021-01-26 05:22:02,-34.9285,138.6007,596,4,592,0,"South Australia, Australia",33.931113008824376,0.6711409395973155
,,Tasmania,Australia,2021-01-26 05:22:02,-42.8821,147.3272,234,13,221,0,"Tasmania, Australia",43.69747899159664,5.555555555555555
,,Victoria,Australia,2021-01-26 05:22:02,-37.8136,144.9631,20443,820,19592,31,"Victoria, Australia",308.34552557353805,4.011152961894047
,,Western Australia,Australia,2021-01-26 05:22:02,-31.9505,115.8605,895,9,873,13,"Western Australia, Australia",34.02265642819129,1.005586592178771
,,,Austria,2021-01-26 05:22:02,47.5162,14.5501,405723,7451,383200,15072,Austria,4504.829898738673,1.8364746391010616
,,,Azerbaijan,2021-01-26 05:22:02,40.1431,47.5769,229032,3093,220565,5374,Azerbaijan,2258.882009630961,1.350466310384575
,,,Bahamas,2021-01-26 05:22:02,25.025885,-78.035889,8133,175,6720,1238,Bahamas,2068.160550085442,2.151727529816796
,,,Bahrain,2021-01-26 05:22:02,26.0275,50.55,100230,369,96761,3100,Bahrain,5890.397353523161,0.36815324753067946
,,,Bangladesh,2021-01-26 05:22:02,23.685,90.3563,532401,8041,476979,47381,Bangladesh,323.27584832836493,1.5103277416834304
,,,Barbados,2021-01-26 05:22:02,13.1939,-59.5432,1401,10,649,742,Barbados,487.5230973201889,0.7137758743754461
,,,Belarus,2021-01-26 05:22:02,53.7098,27.9534,238635,1658,224171,12806,Belarus,2525.41955130956,0.69478492258051
,,Antwerp,Belgium,2021-01-26 05:22:02,51.2195,4.4024,86543,0,0,86543,"Antwerp, Belgium",4657.8930088816605,0.0
,,Brussels,Belgium,2021-01-26 05:22:02,50.8503,4.3517,86841,0,0,86841,"Brussels, Belgium",7185.600500437718,0.0
,,East Flanders,Belgium,2021-01-26 05:22:02,51.0362,3.7373,71710,0,0,71710,"East Flanders, Belgium",4733.133385784363,0.0
,,Flemish Brabant,Belgium,2021-01-26 05:22:02,50.9167,4.5833,52887,0,0,52887,"Flemish Brabant, Belgium",4614.216851702401,0.0
,,Hainaut,Belgium,2021-01-26 05:22:02,50.5257,4.0621,107366,0,0,107366,"Hainaut, Belgium",7987.109454331477,0.0
,,Liege,Belgium,2021-01-26 05:22:02,50.4496,5.8492,100402,0,0,100402,"Liege, Belgium",9069.803575816266,0.0
,,Limburg,Belgium,2021-01-26 05:22:02,50.9739,5.3420000000000005,33198,0,0,33198,"Limburg, Belgium",3798.1895731126892,0.0
,,Luxembourg,Belgium,2021-01-26 05:22:02,50.0547,5.4677,19583,0,0,19583,"Luxembourg, Belgium",68

In [0]:
covid_schema = StructType([
  StructField("FIPS",IntegerType(),True),
  StructField("Admin2",StringType(),True),
  StructField("Province_State",StringType(),True),
  StructField("Country_Region",StringType(),True),
  StructField("Last_Update",TimestampType(),True),
  StructField("Lat", DoubleType(),True),
  StructField("Long_", DoubleType(),True),
  StructField("Confirmed",LongType(),True),
  StructField("Deaths",LongType(),True),
  StructField("Recovered",LongType(),True),
  StructField("Active",LongType(),True),
  StructField("CombinedKey",StringType(),True),
  StructField("Incident_Ratet",DoubleType(),True),
  StructField("Case_Fatality_Ratio",DoubleType(),True) 
])

In [0]:
#global_data = spark.read.csv("dbfs:/databricks-datasets/COVID/CSSEGISandData/csse_covid_19_data/csse_covid_19_daily_reports/01-30-2021.csv", inferSchema=True, header=True)
global_data = spark.read.csv("dbfs:/databricks-datasets/COVID/CSSEGISandData/csse_covid_19_data/csse_covid_19_daily_reports/*.csv", schema=covid_schema, header=True)

##### Show our data

###### Covid DataFrame

In [0]:
global_data.head()

Out[216]: Row(FIPS=None, Admin2=None, Province_State=None, Country_Region='Afghanistan', Last_Update=datetime.datetime(2021, 1, 31, 5, 22, 42), Lat=33.93911, Long_=67.709953, Confirmed=55008, Deaths=2400, Recovered=47609, Active=4999, CombinedKey='Afghanistan', Incident_Ratet=141.3057905550098, Case_Fatality_Ratio=4.363001745200698)

In [0]:
global_data.take(3)

Out[217]: [Row(FIPS=None, Admin2=None, Province_State=None, Country_Region='Afghanistan', Last_Update=datetime.datetime(2021, 1, 31, 5, 22, 42), Lat=33.93911, Long_=67.709953, Confirmed=55008, Deaths=2400, Recovered=47609, Active=4999, CombinedKey='Afghanistan', Incident_Ratet=141.3057905550098, Case_Fatality_Ratio=4.363001745200698),
 Row(FIPS=None, Admin2=None, Province_State=None, Country_Region='Albania', Last_Update=datetime.datetime(2021, 1, 31, 5, 22, 42), Lat=41.1533, Long_=20.1683, Confirmed=77251, Deaths=1369, Recovered=46927, Active=28955, CombinedKey='Albania', Incident_Ratet=2684.376954618111, Case_Fatality_Ratio=1.7721453443968362),
 Row(FIPS=None, Admin2=None, Province_State=None, Country_Region='Algeria', Last_Update=datetime.datetime(2021, 1, 31, 5, 22, 42), Lat=28.0339, Long_=1.6596, Confirmed=107122, Deaths=2888, Recovered=73152, Active=31082, CombinedKey='Algeria', Incident_Ratet=244.2860937195952, Case_Fatality_Ratio=2.695991486342675)]

In [0]:
global_data.show()

+----+------+--------------------+-------------------+-------------------+---------+----------+---------+------+---------+------+--------------------+------------------+-------------------+
FIPS|Admin2| Province_State| Country_Region| Last_Update| Lat| Long_|Confirmed|Deaths|Recovered|Active| CombinedKey| Incident_Ratet|Case_Fatality_Ratio|
+----+------+--------------------+-------------------+-------------------+---------+----------+---------+------+---------+------+--------------------+------------------+-------------------+
null| null| null| Afghanistan|2021-01-31 05:22:42| 33.93911| 67.709953| 55008| 2400| 47609| 4999| Afghanistan| 141.3057905550098| 4.363001745200698|
null| null| null| Albania|2021-01-31 05:22:42| 41.1533| 20.1683| 77251| 1369| 46927| 28955| Albania| 2684.376954618111| 1.7721453443968362|
null| null| null| Algeria|2021-01-31 05:22:42| 28.0339| 1.6596| 107122| 2888| 73152| 31082| Algeria| 244.2860937195952| 2.695991486342675|
null| null| null| Andorra|2021-01-31 05:22:42| 42.5063| 1.5218| 9885| 101| 9045| 739| Andorra|12793.632304406909| 1.0217501264542235|
null| null| null| Angola|2021-01-31 05:22:42| -11.2027| 17.8739| 19782| 464| 18015| 1303| Angola| 60.18937105971387| 2.3455666767768677|
null| null| null|Antigua and Barbuda|2021-01-31 05:22:42| 17.0608| -61.7964| 218| 7| 175| 36| Antigua and Barbuda|222.61253165591046| 3.2110091743119265|
null| null| null| Argentina|2021-01-31 05:22:42| -38.4161| -63.6167| 1922264| 47931| 1712868|161465| Argentina| 4253.19383268928| 2.493466037963568|
null| null| null| Armenia|2021-01-31 05:22:42| 40.0691| 45.0382| 166901| 3071| 157314| 6516| Armenia| 5632.393526802136| 1.8400129418038238|
null| null|Australian Capita...| Australia|2021-01-31 05:22:42| -35.4735| 149.0124| 118| 3| 115| 0|Australian Capita...|27.563653352020555| 2.542372881355932|
null| null| New South Wales| Australia|2021-01-31 05:22:42| -33.8688| 151.2093| 5104| 54| 3255| 1795|New South Wales, ...| 62.87262872628726| 1.0579937304075235|
null| null| Northern Territory| Australia|2021-01-31 05:22:42| -12.4634| 130.8456| 98| 0| 93| 5|Northern Territor...| 39.90228013029316| 0.0|
null| null| Queensland| Australia|2021-01-31 05:22:42| -27.4698| 153.0251| 1309| 6| 1295| 8|Queensland, Austr...| 25.58889649105659|0.45836516424751717|
null| null| South Australia| Australia|2021-01-31 05:22:42| -34.9285| 138.6007| 597| 4| 592| 1|South Australia, ...|33.988044406490175| 0.6700167504187605|
null| null| Tasmania| Australia|2021-01-31 05:22:42| -42.8821| 147.3272| 234| 13| 221| 0| Tasmania, Australia| 43.69747899159664| 5.555555555555555|
null| null| Victoria| Australia|2021-01-31 05:22:42| -37.8136| 144.9631| 20448| 820| 19608| 20| Victoria, Australia|308.42094149233014| 4.010172143974961|
null| null| Western Australia| Australia|2021-01-31 05:22:42| -31.9505| 115.8605| 903| 9| 878| 16|Western Australia...| 34.32676955827568| 0.9966777408637874|
null| null| null| Austria|2021-01-31 05:22:42| 47.5162| 14.5501| 413208| 7703| 391076| 14429| Austria| 4587.937466690353| 1.8641943040793014|
null| null| null| Azerbaijan|2021-01-31 05:22:42| 40.1431| 47.5769| 230066| 3126| 222960| 3980| Azerbaijan| 2269.080078014237| 1.358740535324646|
null| null| null| Bahamas|2021-01-31 05:22:42|25.025885|-78.035889| 8174| 176| 6771| 1227| Bahamas| 2078.586540808853| 2.1531685833129437|
null| null| null| Bahrain|2021-01-31 05:22:42| 26.0275| 50.55| 102626| 372| 98160| 4094| Bahrain| 6031.207410981421|0.36248124257010894|
+----+------+--------------------+-------------------+-------------------+---------+----------+---------+------+---------+------+--------------------+------------------+-------------------+
only showing top 20 rows

##### Make our data more easy to work with

In [0]:
global_data = global_data.withColumn("date", to_date(col("Last_Update")))
global_data.show()

+----+------+--------------------+-------------------+-------------------+---------+----------+---------+------+---------+------+--------------------+------------------+-------------------+----------+
FIPS|Admin2| Province_State| Country_Region| Last_Update| Lat| Long_|Confirmed|Deaths|Recovered|Active| CombinedKey| Incident_Ratet|Case_Fatality_Ratio| date|
+----+------+--------------------+-------------------+-------------------+---------+----------+---------+------+---------+------+--------------------+------------------+-------------------+----------+
null| null| null| Afghanistan|2021-01-31 05:22:42| 33.93911| 67.709953| 55008| 2400| 47609| 4999| Afghanistan| 141.3057905550098| 4.363001745200698|2021-01-31|
null| null| null| Albania|2021-01-31 05:22:42| 41.1533| 20.1683| 77251| 1369| 46927| 28955| Albania| 2684.376954618111| 1.7721453443968362|2021-01-31|
null| null| null| Algeria|2021-01-31 05:22:42| 28.0339| 1.6596| 107122| 2888| 73152| 31082| Algeria| 244.2860937195952| 2.695991486342675|2021-01-31|
null| null| null| Andorra|2021-01-31 05:22:42| 42.5063| 1.5218| 9885| 101| 9045| 739| Andorra|12793.632304406909| 1.0217501264542235|2021-01-31|
null| null| null| Angola|2021-01-31 05:22:42| -11.2027| 17.8739| 19782| 464| 18015| 1303| Angola| 60.18937105971387| 2.3455666767768677|2021-01-31|
null| null| null|Antigua and Barbuda|2021-01-31 05:22:42| 17.0608| -61.7964| 218| 7| 175| 36| Antigua and Barbuda|222.61253165591046| 3.2110091743119265|2021-01-31|
null| null| null| Argentina|2021-01-31 05:22:42| -38.4161| -63.6167| 1922264| 47931| 1712868|161465| Argentina| 4253.19383268928| 2.493466037963568|2021-01-31|
null| null| null| Armenia|2021-01-31 05:22:42| 40.0691| 45.0382| 166901| 3071| 157314| 6516| Armenia| 5632.393526802136| 1.8400129418038238|2021-01-31|
null| null|Australian Capita...| Australia|2021-01-31 05:22:42| -35.4735| 149.0124| 118| 3| 115| 0|Australian Capita...|27.563653352020555| 2.542372881355932|2021-01-31|
null| null| New South Wales| Australia|2021-01-31 05:22:42| -33.8688| 151.2093| 5104| 54| 3255| 1795|New South Wales, ...| 62.87262872628726| 1.0579937304075235|2021-01-31|
null| null| Northern Territory| Australia|2021-01-31 05:22:42| -12.4634| 130.8456| 98| 0| 93| 5|Northern Territor...| 39.90228013029316| 0.0|2021-01-31|
null| null| Queensland| Australia|2021-01-31 05:22:42| -27.4698| 153.0251| 1309| 6| 1295| 8|Queensland, Austr...| 25.58889649105659|0.45836516424751717|2021-01-31|
null| null| South Australia| Australia|2021-01-31 05:22:42| -34.9285| 138.6007| 597| 4| 592| 1|South Australia, ...|33.988044406490175| 0.6700167504187605|2021-01-31|
null| null| Tasmania| Australia|2021-01-31 05:22:42| -42.8821| 147.3272| 234| 13| 221| 0| Tasmania, Australia| 43.69747899159664| 5.555555555555555|2021-01-31|
null| null| Victoria| Australia|2021-01-31 05:22:42| -37.8136| 144.9631| 20448| 820| 19608| 20| Victoria, Australia|308.42094149233014| 4.010172143974961|2021-01-31|
null| null| Western Australia| Australia|2021-01-31 05:22:42| -31.9505| 115.8605| 903| 9| 878| 16|Western Australia...| 34.32676955827568| 0.9966777408637874|2021-01-31|
null| null| null| Austria|2021-01-31 05:22:42| 47.5162| 14.5501| 413208| 7703| 391076| 14429| Austria| 4587.937466690353| 1.8641943040793014|2021-01-31|
null| null| null| Azerbaijan|2021-01-31 05:22:42| 40.1431| 47.5769| 230066| 3126| 222960| 3980| Azerbaijan| 2269.080078014237| 1.358740535324646|2021-01-31|
null| null| null| Bahamas|2021-01-31 05:22:42|25.025885|-78.035889| 8174| 176| 6771| 1227| Bahamas| 2078.586540808853| 2.1531685833129437|2021-01-31|
null| null| null| Bahrain|2021-01-31 05:22:42| 26.0275| 50.55| 102626| 372| 98160| 4094| Bahrain| 6031.207410981421|0.36248124257010894|2021-01-31|
+----+------+--------------------+-------------------+-------------------+---------+----------+---------+------+---------+------+--------------------+------------------+-------------------+----------+
only showing top 20 rows

In [0]:
global_data.where(col("date").isNull()).count()

Out[220]: 29350

In [0]:
global_data = global_data.na.drop(subset=["Confirmed", "Active", "date"])

In [0]:
global_data = global_data.drop("Lat", "Long_", "CombinedKey", "Last_Update" )
global_data.show()

+----+------+--------------------+-------------------+---------+------+---------+------+------------------+-------------------+----------+
FIPS|Admin2| Province_State| Country_Region|Confirmed|Deaths|Recovered|Active| Incident_Ratet|Case_Fatality_Ratio| date|
+----+------+--------------------+-------------------+---------+------+---------+------+------------------+-------------------+----------+
null| null| null| Afghanistan| 55008| 2400| 47609| 4999| 141.3057905550098| 4.363001745200698|2021-01-31|
null| null| null| Albania| 77251| 1369| 46927| 28955| 2684.376954618111| 1.7721453443968362|2021-01-31|
null| null| null| Algeria| 107122| 2888| 73152| 31082| 244.2860937195952| 2.695991486342675|2021-01-31|
null| null| null| Andorra| 9885| 101| 9045| 739|12793.632304406909| 1.0217501264542235|2021-01-31|
null| null| null| Angola| 19782| 464| 18015| 1303| 60.18937105971387| 2.3455666767768677|2021-01-31|
null| null| null|Antigua and Barbuda| 218| 7| 175| 36|222.61253165591046| 3.2110091743119265|2021-01-31|
null| null| null| Argentina| 1922264| 47931| 1712868|161465| 4253.19383268928| 2.493466037963568|2021-01-31|
null| null| null| Armenia| 166901| 3071| 157314| 6516| 5632.393526802136| 1.8400129418038238|2021-01-31|
null| null|Australian Capita...| Australia| 118| 3| 115| 0|27.563653352020555| 2.542372881355932|2021-01-31|
null| null| New South Wales| Australia| 5104| 54| 3255| 1795| 62.87262872628726| 1.0579937304075235|2021-01-31|
null| null| Northern Territory| Australia| 98| 0| 93| 5| 39.90228013029316| 0.0|2021-01-31|
null| null| Queensland| Australia| 1309| 6| 1295| 8| 25.58889649105659|0.45836516424751717|2021-01-31|
null| null| South Australia| Australia| 597| 4| 592| 1|33.988044406490175| 0.6700167504187605|2021-01-31|
null| null| Tasmania| Australia| 234| 13| 221| 0| 43.69747899159664| 5.555555555555555|2021-01-31|
null| null| Victoria| Australia| 20448| 820| 19608| 20|308.42094149233014| 4.010172143974961|2021-01-31|
null| null| Western Australia| Australia| 903| 9| 878| 16| 34.32676955827568| 0.9966777408637874|2021-01-31|
null| null| null| Austria| 413208| 7703| 391076| 14429| 4587.937466690353| 1.8641943040793014|2021-01-31|
null| null| null| Azerbaijan| 230066| 3126| 222960| 3980| 2269.080078014237| 1.358740535324646|2021-01-31|
null| null| null| Bahamas| 8174| 176| 6771| 1227| 2078.586540808853| 2.1531685833129437|2021-01-31|
null| null| null| Bahrain| 102626| 372| 98160| 4094| 6031.207410981421|0.36248124257010894|2021-01-31|
+----+------+--------------------+-------------------+---------+------+---------+------+------------------+-------------------+----------+
only showing top 20 rows

In [0]:
global_data = global_data.withColumnRenamed("Admin2", "county")
global_data.show()

+----+------+--------------------+-------------------+---------+------+---------+------+------------------+-------------------+----------+
FIPS|county| Province_State| Country_Region|Confirmed|Deaths|Recovered|Active| Incident_Ratet|Case_Fatality_Ratio| date|
+----+------+--------------------+-------------------+---------+------+---------+------+------------------+-------------------+----------+
null| null| null| Afghanistan| 55008| 2400| 47609| 4999| 141.3057905550098| 4.363001745200698|2021-01-31|
null| null| null| Albania| 77251| 1369| 46927| 28955| 2684.376954618111| 1.7721453443968362|2021-01-31|
null| null| null| Algeria| 107122| 2888| 73152| 31082| 244.2860937195952| 2.695991486342675|2021-01-31|
null| null| null| Andorra| 9885| 101| 9045| 739|12793.632304406909| 1.0217501264542235|2021-01-31|
null| null| null| Angola| 19782| 464| 18015| 1303| 60.18937105971387| 2.3455666767768677|2021-01-31|
null| null| null|Antigua and Barbuda| 218| 7| 175| 36|222.61253165591046| 3.2110091743119265|2021-01-31|
null| null| null| Argentina| 1922264| 47931| 1712868|161465| 4253.19383268928| 2.493466037963568|2021-01-31|
null| null| null| Armenia| 166901| 3071| 157314| 6516| 5632.393526802136| 1.8400129418038238|2021-01-31|
null| null|Australian Capita...| Australia| 118| 3| 115| 0|27.563653352020555| 2.542372881355932|2021-01-31|
null| null| New South Wales| Australia| 5104| 54| 3255| 1795| 62.87262872628726| 1.0579937304075235|2021-01-31|
null| null| Northern Territory| Australia| 98| 0| 93| 5| 39.90228013029316| 0.0|2021-01-31|
null| null| Queensland| Australia| 1309| 6| 1295| 8| 25.58889649105659|0.45836516424751717|2021-01-31|
null| null| South Australia| Australia| 597| 4| 592| 1|33.988044406490175| 0.6700167504187605|2021-01-31|
null| null| Tasmania| Australia| 234| 13| 221| 0| 43.69747899159664| 5.555555555555555|2021-01-31|
null| null| Victoria| Australia| 20448| 820| 19608| 20|308.42094149233014| 4.010172143974961|2021-01-31|
null| null| Western Australia| Australia| 903| 9| 878| 16| 34.32676955827568| 0.9966777408637874|2021-01-31|
null| null| null| Austria| 413208| 7703| 391076| 14429| 4587.937466690353| 1.8641943040793014|2021-01-31|
null| null| null| Azerbaijan| 230066| 3126| 222960| 3980| 2269.080078014237| 1.358740535324646|2021-01-31|
null| null| null| Bahamas| 8174| 176| 6771| 1227| 2078.586540808853| 2.1531685833129437|2021-01-31|
null| null| null| Bahrain| 102626| 372| 98160| 4094| 6031.207410981421|0.36248124257010894|2021-01-31|
+----+------+--------------------+-------------------+---------+------+---------+------+------------------+-------------------+----------+
only showing top 20 rows

In [0]:
global_data = global_data.filter(column("Active") >= 0)

In [0]:
global_data.persist()

Out[250]: DataFrame[FIPS: int, county: string, Province_State: string, Country_Region: string, Confirmed: bigint, Deaths: bigint, Recovered: bigint, Active: bigint, Incident_Ratet: double, Case_Fatality_Ratio: double, date: date]

##### Counting number of rows

In [0]:
global_data.count()

Out[251]: 1157350

In [0]:
global_data.sample(0.00003).show()

+-----+-----------+--------------+--------------+---------+------+---------+------+------------------+-------------------+----------+
 FIPS| county|Province_State|Country_Region|Confirmed|Deaths|Recovered|Active| Incident_Ratet|Case_Fatality_Ratio| date|
+-----+-----------+--------------+--------------+---------+------+---------+------+------------------+-------------------+----------+
 null| null| Shanghai| China| 1644.0| 7| 1532| 105| 6.782178217821782| 0.4257907542579075|2021-01-25|
21073| Franklin| Kentucky| US| 2914.0| 24| 0| 2890| 5714.733972661842| 0.8236101578586136|2021-01-28|
 null| null| null| Mauritius| 527.0| 10| 496| 21|41.438408136081534| 1.8975332068311195|2021-01-03|
29149| Oregon| Missouri| US| 526.0| 3| 0| 523|4995.7260898470895| 0.5703422053231939|2020-12-30|
42121| Venango| Pennsylvania| US| 2125.0| 35| 0| 2090| 4193.968579774217| 1.6470588235294117|2020-12-24|
 6099| Stanislaus| California| US| 33158.0| 533| 0| 30016| 5547.7063886972| 1.7447379619627483|2020-12-22|
 null| null| Vasterbotten| Sweden| 2152.0| 32| 0| 2120| 791.9451232078194| 1.486988847583643|2020-11-11|
18013| Brown| Indiana| US| 146.0| 3| 0| 143| 967.3999469917836| 2.0547945205479454|2020-10-26|
29053| Cooper| Missouri| US| 609.0| 3| 0| 606| 3438.929357953583|0.49261083743842365|2020-10-27|
12035| Flagler| Florida| US| 1927.0| 35| 0| 1892|1674.4727626628198| 1.8162947586922678|2020-10-17|
 4015| Mohave| Arizona| US| 3606.0| 206| 0| 3400|1699.4924144951715| 5.712701053799224|2020-08-31|
12123| Taylor| Florida| US| 123.0| 1| 0| 122| 570.2628772775743| 0.8130081300813008|2020-07-11|
28147| Walthall| Mississippi| US| 172.0| 4| 0| 168|1203.9759204815903| 2.3255813953488373|2020-07-02|
51131|Northampton| Virginia| US| 262.0| 27| 0| 235| 2237.403928266439| 10.305343511450381|2020-06-15|
 null| null| Sergipe| Brazil| 76073.0| 1987| 70008| 4078| 3309.398024| 2.611964823|2020-09-22|
 null| null| null| Burundi| 85.0| 1| 45| 39|0.7148395046549088| 1.1764705882352942|2020-06-12|
31017| Brown| Nebraska| US| 17.0| 0| 0| 17| 575.2961083| 0.0|2020-09-20|
55047| Green Lake| Wisconsin| US| 169.0| 0| 0| 169| 893.5652726| 0.0|2020-09-19|
37071| Gaston|North Carolina| US| 288.0| 7| 0| 281|128.26850874497282| 2.4305555555555554|2020-06-03|
29063| DeKalb| Missouri| US| 6.0| 0| 0| 6| 47.82019606280386| 0.0|2020-06-02|
+-----+-----------+--------------+--------------+---------+------+---------+------+------------------+-------------------+----------+
only showing top 20 rows

##### Simple Select

In [0]:
global_data.select("Country_Region", "Confirmed").show(5)
global_data.select(col("Country_Region"), col("Confirmed")).show(5)
global_data.select(column("Country_Region"), column("Confirmed")).show(5)
global_data.select(global_data["Country_Region"], global_data["Confirmed"]).show(5)


+--------------+---------+
Country_Region|Confirmed|
+--------------+---------+
 Afghanistan| 55008|
 Albania| 77251|
 Algeria| 107122|
 Andorra| 9885|
 Angola| 19782|
+--------------+---------+
only showing top 5 rows

+--------------+---------+
Country_Region|Confirmed|
+--------------+---------+
 Afghanistan| 55008|
 Albania| 77251|
 Algeria| 107122|
 Andorra| 9885|
 Angola| 19782|
+--------------+---------+
only showing top 5 rows

+--------------+---------+
Country_Region|Confirmed|
+--------------+---------+
 Afghanistan| 55008|
 Albania| 77251|
 Algeria| 107122|
 Andorra| 9885|
 Angola| 19782|
+--------------+---------+
only showing top 5 rows

+--------------+---------+
Country_Region|Confirmed|
+--------------+---------+
 Afghanistan| 55008|
 Albania| 77251|
 Algeria| 107122|
 Andorra| 9885|
 Angola| 19782|
+--------------+---------+
only showing top 5 rows

In [0]:
global_data.select("date", col("Confirmed") + 1).show(5)
#global_data.select("date", "Confirmed" + 1).show(5)

+----------+---------------+
 date|(Confirmed + 1)|
+----------+---------------+
2021-01-31| 55009|
2021-01-31| 77252|
2021-01-31| 107123|
2021-01-31| 9886|
2021-01-31| 19783|
+----------+---------------+
only showing top 5 rows

In [0]:
global_data.select("Country_Region","date", "Confirmed")\
        .withColumn("grater than 1000", (col("confirmed") > 1000)).show(100)
#global_data.select("date", "Confirmed").withColumn("grater than 1000", ("Confirmed" > 1000)).show(5)

+--------------------+----------+---------+----------------+
 Country_Region| date|Confirmed|grater than 1000|
+--------------------+----------+---------+----------------+
 Afghanistan|2021-01-31| 55008| true|
 Albania|2021-01-31| 77251| true|
 Algeria|2021-01-31| 107122| true|
 Andorra|2021-01-31| 9885| true|
 Angola|2021-01-31| 19782| true|
 Antigua and Barbuda|2021-01-31| 218| false|
 Argentina|2021-01-31| 1922264| true|
 Armenia|2021-01-31| 166901| true|
 Australia|2021-01-31| 118| false|
 Australia|2021-01-31| 5104| true|
 Australia|2021-01-31| 98| false|
 Australia|2021-01-31| 1309| true|
 Australia|2021-01-31| 597| false|
 Australia|2021-01-31| 234| false|
 Australia|2021-01-31| 20448| true|
 Australia|2021-01-31| 903| false|
 Austria|2021-01-31| 413208| true|
 Azerbaijan|2021-01-31| 230066| true|
 Bahamas|2021-01-31| 8174| true|
 Bahrain|2021-01-31| 102626| true|
 Bangladesh|2021-01-31| 534770| true|
 Barbados|2021-01-31| 1523| true|
 Belarus|2021-01-31| 246570| true|
 Belgium|2021-01-31| 88269| true|
 Belgium|2021-01-31| 87956| true|
 Belgium|2021-01-31| 73394| true|
 Belgium|2021-01-31| 54012| true|
 Belgium|2021-01-31| 108969| true|
 Belgium|2021-01-31| 101467| true|
 Belgium|2021-01-31| 34473| true|
 Belgium|2021-01-31| 19947| true|
 Belgium|2021-01-31| 37488| true|
 Belgium|2021-01-31| 27371| true|
 Belgium|2021-01-31| 61931| true|
 Belize|2021-01-31| 11908| true|
 Benin|2021-01-31| 3786| true|
 Bhutan|2021-01-31| 858| false|
 Bolivia|2021-01-31| 215397| true|
Bosnia and Herzeg...|2021-01-31| 121497| true|
 Botswana|2021-01-31| 21293| true|
 Brazil|2021-01-31| 48347| true|
 Brazil|2021-01-31| 117287| true|
 Brazil|2021-01-31| 76884| true|
 Brazil|2021-01-31| 265994| true|
 Brazil|2021-01-31| 585958| true|
 Brazil|2021-01-31| 370300| true|
 Brazil|2021-01-31| 276572| true|
 Brazil|2021-01-31| 293030| true|
 Brazil|2021-01-31| 350349| true|
 Brazil|2021-01-31| 207415| true|
 Brazil|2021-01-31| 215392| true|
 Brazil|2021-01-31| 160623| true|
 Brazil|2021-01-31| 729968| true|
 Brazil|2021-01-31| 328800| true|
 Brazil|2021-01-31| 190480| true|
 Brazil|2021-01-31| 547430| true|
 Brazil|2021-01-31| 260679| true|
 Brazil|2021-01-31| 158584| true|
 Brazil|2021-01-31| 140032| true|
 Brazil|2021-01-31| 546708| true|
 Brazil|2021-01-31| 520730| true|
 Brazil|2021-01-31| 123957| true|
 Brazil|2021-01-31| 73909| true|
 Brazil|2021-01-31| 575616| true|
 Brazil|2021-01-31| 1773024| true|
 Brazil|2021-01-31| 136799| true|
 Brazil|2021-01-31| 102108| true|
 Brunei|2021-01-31| 180| false|
 Bulgaria|2021-01-31| 218618| true|
 Burkina Faso|2021-01-31| 10580| true|
 Burma|2021-01-31| 139864| true|
 Burundi|2021-01-31| 1613| true|
 Cabo Verde|2021-01-31| 13981| true|
 Cambodia|2021-01-31| 465| false|
 Cameroon|2021-01-31| 29617| true|
 Canada|2021-01-31| 123747| true|
 Canada|2021-01-31| 66779| true|
 Canada|2020-12-21| 13| false|
 Canada|2021-01-31| 29446| true|
 Canada|2021-01-31| 1230| true|
 Canada|2021-01-31| 408| false|
 Canada|2021-01-31| 35| false|
 Canada|2021-01-31| 1570| true|
 Canada|2021-01-31| 284| false|
 Canada|2021-01-31| 271060| true|
 Canada|2021-01-31| 111| false|
 Canada|2021-01-31| 261360| true|
 Canada|2021-01-31| 13| false|
 Canada|2021-01-31| 22646| true|
 Canada|2021-01-31| 70| false|
Central African R...|2021-01-31| 4981| true|
 Chad|2021-01-31| 3347| true|
 Chile|2021-01-31| 31159| true|
 Chile|2021-01-31| 31538| true|
 Chile|2021-01-31| 12855| true|
 Chile|2021-01-31| 10081| true|
 Chile|2021-01-31| 2478| true|
 Chile|2021-01-31| 60569| true|
 Chile|2021-01-31| 16839| true|
 Chile|2021-01-31| 41573| true|
+--------------------+----------+---------+----------------+
only showing top 100 rows

##### Get the oldest informations

In [0]:
#global_data.orderBy(global_data["Date"]).show()
global_data.sort(global_data["date"]).show()

+----+------+--------------+--------------+---------+------+---------+------+--------------+-------------------+----------+
FIPS|county|Province_State|Country_Region|Confirmed|Deaths|Recovered|Active|Incident_Ratet|Case_Fatality_Ratio| date|
+----+------+--------------+--------------+---------+------+---------+------+--------------+-------------------+----------+
null| null| Tibet| China| 1| 0| 1| 0| null| null|2020-02-23|
null| null| Qinghai| China| 18| 0| 18| 0| null| null|2020-02-23|
null| null| Tibet| China| 1| 0| 1| 0| null| null|2020-02-23|
null| null| Tibet| China| 1| 0| 1| 0| null| null|2020-02-23|
null| null| Tibet| China| 1| 0| 1| 0| null| null|2020-02-23|
null| null| Tibet| China| 1| 0| 1| 0| null| null|2020-02-23|
null| null| Tibet| China| 1| 0| 1| 0| null| null|2020-02-23|
null| null| Tibet| China| 1| 0| 1| 0| null| null|2020-02-23|
null| null| Tibet| China| 1| 0| 1| 0| null| null|2020-02-23|
null| null| Qinghai| China| 18| 0| 18| 0| null| null|2020-02-23|
null| null| Qinghai| China| 18| 0| 18| 0| null| null|2020-02-23|
null| null| Qinghai| China| 18| 0| 18| 0| null| null|2020-02-23|
null| null| Qinghai| China| 18| 0| 18| 0| null| null|2020-02-23|
null| null| Tibet| China| 1| 0| 1| 0| null| null|2020-02-23|
null| null| Tibet| China| 1| 0| 1| 0| null| null|2020-02-23|
null| null| Tibet| China| 1| 0| 1| 0| null| null|2020-02-23|
null| null| Qinghai| China| 18| 0| 18| 0| null| null|2020-02-23|
null| null| Tibet| China| 1| 0| 1| 0| null| null|2020-02-23|
null| null| Qinghai| China| 18| 0| 18| 0| null| null|2020-02-23|
null| null| Qinghai| China| 18| 0| 18| 0| null| null|2020-02-23|
+----+------+--------------+--------------+---------+------+---------+------+--------------+-------------------+----------+
only showing top 20 rows

##### Get informations about the countries

In [0]:
#global_data.select("Country_Region").distinct().count()
global_data.select(col("Country_Region")).dropDuplicates(["Country_Region"]).show(193)


+--------------------+
 Country_Region|
+--------------------+
 Chad|
 Paraguay|
 Russia|
 Yemen|
 Senegal|
 Cabo Verde|
 Sweden|
 Guyana|
 Burma|
 Eritrea|
 Philippines|
 Djibouti|
 Malaysia|
 Singapore|
 Fiji|
 Turkey|
 Malawi|
 Iraq|
 Germany|
 Comoros|
 Afghanistan|
 Cambodia|
 Jordan|
 Maldives|
 Rwanda|
 Sudan|
 France|
 Greece|
 Kosovo|
 MS Zaandam|
 Sri Lanka|
 Dominica|
 Algeria|
 Equatorial Guinea|
 Togo|
 Slovakia|
 Argentina|
 Angola|
 Belgium|
 San Marino|
 Ecuador|
 Qatar|
 Lesotho|
 Albania|
 Madagascar|
 Finland|
 Ghana|
 Nicaragua|
 Benin|
 Peru|
 Sierra Leone|
 Bahamas|
 China|
 India|
 Belarus|
 Kuwait|
 Malta|
 Timor-Leste|
Sao Tome and Prin...|
 Marshall Islands|
 Somalia|
 Chile|
 Tajikistan|
 Croatia|
 Burundi|
 Andorra|
 Bolivia|
 Nigeria|
 Gabon|
 Italy|
 Suriname|
 Lithuania|
 Norway|
 Spain|
 Cuba|
 Mauritania|
 Czechia|
 Congo (Kinshasa)|
Central African R...|
 Denmark|
 Niger|
 Bangladesh|
 Barbados|
 Iran|
 US|
 Ireland|
 Liechtenstein|
 Laos|
 Thailand|
 Bhutan|
 Monaco|
 Morocco|
 Panama|
 Ukraine|
 Venezuela|
 Iceland|
 Israel|
 Holy See|
Saint Kitts and N...|
 Oman|
 Taiwan*|
 Cyprus|
 Uruguay|
 Cote d'Ivoire|
 Mexico|
 Estonia|
 Georgia|
 Montenegro|
 Zimbabwe|
 Indonesia|
Saint Vincent and...|
 Guatemala|
 Mongolia|
 Azerbaijan|
 Libya|
 Grenada|
 Armenia|
 Liberia|
 Tunisia|
 Honduras|
 Syria|
 Trinidad and Tobago|
 Congo (Brazzaville)|
 Saudi Arabia|
 Uganda|
 Namibia|
 Switzerland|
 Zambia|
 Ethiopia|
 Eswatini|
 Jamaica|
 Latvia|
 Brunei|
 South Sudan|
United Arab Emirates|
 Saint Lucia|
 Guinea|
 Canada|
 Seychelles|
 Kyrgyzstan|
 North Macedonia|
 Samoa|
 Uzbekistan|
 Mozambique|
 Korea, South|
 Belize|
 Brazil|
 Gambia|
 Kenya|
 West Bank and Gaza|
 Lebanon|
 Antigua and Barbuda|
 Slovenia|
 Dominican Republic|
 Botswana|
 Japan|
 Tanzania|
 Luxembourg|
 New Zealand|
Bosnia and Herzeg...|
 Haiti|
 Poland|
 Portugal|
 Australia|
 Cameroon|
 Papua New Guinea|
 Guinea-Bissau|
 Romania|
 Bulgaria|
 Solomon Islands|
 Austria|
 Nepal|
 Costa Rica|
 Egypt|
 El Salvador|
 Kazakhstan|
 Serbia|
 Burkina Faso|
 South Africa|
 Bahrain|
 Micronesia|
 Colombia|
 Hungary|
 Pakistan|
 Vanuatu|
 Mauritius|
 Moldova|
 United Kingdom|
 Vietnam|
 Diamond Princess|
 Mali|
 Netherlands|
 Cruise Ship|
+--------------------+

##### Get the informations from Poland

In [0]:
global_data.filter(global_data["Country_Region"] == "Poland").show()
#global_data.where(cglobal_data["Country_Region"] == "Poland").show()

+----+------+--------------+--------------+---------+------+---------+------+------------------+-------------------+----------+
FIPS|county|Province_State|Country_Region|Confirmed|Deaths|Recovered|Active| Incident_Ratet|Case_Fatality_Ratio| date|
+----+------+--------------+--------------+---------+------+---------+------+------------------+-------------------+----------+
null| null| null| Poland| 1508674| 37082| 1264574|207018|3986.2862203888567| 2.4579200012726408|2021-01-31|
null| null| null| Poland| 1502810| 36780| 1257352|208678|3970.7920961470654| 2.4474151755710967|2021-01-30|
null| null| null| Poland| 1496665| 36443| 1250892|209330| 3954.555501081273| 2.4349470322350024|2021-01-29|
null| null| null| Poland| 1489512| 36054| 1246267|207191| 3935.655523130807| 2.4205243059471826|2021-01-28|
null| null| null| Poland| 1482722| 35665| 1243575|203482| 3917.714680088214| 2.4053733606164878|2021-01-27|
null| null| null| Poland| 1478119| 35401| 1237736|204982|3905.5524266971893| 2.3950033792949013|2021-01-26|
null| null| null| Poland| 1475445| 35363| 1230446|209636| 3898.487063767014| 2.396768432574579|2021-01-25|
null| null| null| Poland| 1470879| 35253| 1222782|212844|3886.4225734382253| 2.396730118520966|2021-01-24|
null| null| null| Poland| 1464448| 34908| 1215732|213808| 3869.430296323805| 2.3836967922384407|2021-01-23|
null| null| null| Poland| 1457755| 34561| 1215732|207462| 3851.745751038964| 2.37083734921163|2021-01-22|
null| null| null| Poland| 1450747| 34141| 1207359|209247| 3833.228898602662| 2.3533393486252256|2021-01-21|
null| null| null| Poland| 1443804| 33698| 1197034|213072|3814.8837920865035| 2.3339733093965664|2021-01-20|
null| null| null| Poland| 1438914| 33407| 1187719|217788| 3801.963214401926| 2.3216814903461915|2021-01-19|
null| null| null| Poland| 1435582| 33355| 1181326|220901|3793.1592543109223| 2.323447911718035|2021-01-18|
null| null| null| Poland| 1429612| 33213| 1173087|223312| 3777.385052107052| 2.3232177681776593|2021-01-17|
null| null| null| Poland| 1422320| 32844| 1164650|224826| 3758.117802111973| 2.309184993531695|2021-01-16|
null| null| null| Poland| 1414362| 32456| 1155823|226083|3737.0908169966633| 2.294744909719011|2021-01-15|
null| null| null| Poland| 1404905| 32074| 1147123|225708|3712.1031067383724| 2.2830013417277324|2021-01-14|
null| null| null| Poland| 1395779| 31593| 1138126|226060|3687.9899795503447| 2.263467210783369|2021-01-13|
null| null| null| Poland| 1390385| 31264| 1130460|228661| 3673.737710423432| 2.2485858233510863|2021-01-12|
+----+------+--------------+--------------+---------+------+---------+------+------------------+-------------------+----------+
only showing top 20 rows

In [0]:
new_df = global_data.select("*").sort(global_data["date"].desc()).filter(global_data["Country_Region"] == "Poland").select("Country_Region", "date", "Confirmed", "Active").filter(year(col("date")) == 2020)
new_df.show()


+--------------+----------+---------+------+
Country_Region| date|Confirmed|Active|
+--------------+----------+---------+------+
 Poland|2020-12-31| 1281414|227506|
 Poland|2020-12-30| 1268634|227175|
 Poland|2020-12-29| 1261010|228487|
 Poland|2020-12-28| 1257799|231071|
 Poland|2020-12-27| 1253957|234515|
 Poland|2020-12-26| 1249079|237404|
 Poland|2020-12-25| 1239998|237370|
 Poland|2020-12-24| 1226883|235450|
 Poland|2020-12-23| 1214525|234470|
 Poland|2020-12-22| 1207333|235842|
 Poland|2020-12-21| 1202700|239034|
 Poland|2020-12-20| 1194110|241133|
 Poland|2020-12-19| 1182864|242517|
 Poland|2020-12-18| 1171854|244160|
 Poland|2020-12-17| 1159901|243337|
 Poland|2020-12-16| 1147446|244389|
 Poland|2020-12-15| 1140572|248457|
 Poland|2020-12-14| 1135676|253917|
 Poland|2020-12-13| 1126700|263483|
 Poland|2020-12-12| 1115201|277671|
+--------------+----------+---------+------+
only showing top 20 rows

In [0]:
new_df.explain(True)

== Parsed Logical Plan ==
'Filter (year('date) = 2020)
+- Project [Country_Region#9754, date#9878, Confirmed#9758L, Active#9761L]
 +- Filter (Country_Region#9754 = Poland)
 +- Sort [date#9878 DESC NULLS LAST], true
 +- Project [FIPS#9751, county#10072, Province_State#9753, Country_Region#9754, Confirmed#9758L, Deaths#9759L, Recovered#9760L, Active#9761L, Incident_Ratet#9763, Case_Fatality_Ratio#9764, date#9878]
 +- Filter (Active#9761L >= cast(0 as bigint))
 +- Filter (Active#9761L >= cast(0 as bigint))
 +- Project [FIPS#9751, Admin2#9752 AS county#10072, Province_State#9753, Country_Region#9754, Confirmed#9758L, Deaths#9759L, Recovered#9760L, Active#9761L, Incident_Ratet#9763, Case_Fatality_Ratio#9764, date#9878]
 +- Project [FIPS#9751, Admin2#9752, Province_State#9753, Country_Region#9754, Confirmed#9758L, Deaths#9759L, Recovered#9760L, Active#9761L, Incident_Ratet#9763, Case_Fatality_Ratio#9764, date#9878]
 +- Filter AtLeastNNulls(n, Confirmed#9758L,Deaths#9759L,Recovered#9760L,Active#9761L,date#9878)
 +- Project [FIPS#9751, Admin2#9752, Province_State#9753, Country_Region#9754, Last_Update#9755, Lat#9756, Long_#9757, Confirmed#9758L, Deaths#9759L, Recovered#9760L, Active#9761L, CombinedKey#9762, Incident_Ratet#9763, Case_Fatality_Ratio#9764, to_date('Last_Update, None) AS date#9878]
 +- Relation[FIPS#9751,Admin2#9752,Province_State#9753,Country_Region#9754,Last_Update#9755,Lat#9756,Long_#9757,Confirmed#9758L,Deaths#9759L,Recovered#9760L,Active#9761L,CombinedKey#9762,Incident_Ratet#9763,Case_Fatality_Ratio#9764] csv

== Analyzed Logical Plan ==
Country_Region: string, date: date, Confirmed: bigint, Active: bigint
Filter (year(date#9878) = 2020)
+- Project [Country_Region#9754, date#9878, Confirmed#9758L, Active#9761L]
 +- Filter (Country_Region#9754 = Poland)
 +- Sort [date#9878 DESC NULLS LAST], true
 +- Project [FIPS#9751, county#10072, Province_State#9753, Country_Region#9754, Confirmed#9758L, Deaths#9759L, Recovered#9760L, Active#9761L, Incident_Ratet#9763, Case_Fatality_Ratio#9764, date#9878]
 +- Filter (Active#9761L >= cast(0 as bigint))
 +- Filter (Active#9761L >= cast(0 as bigint))
 +- Project [FIPS#9751, Admin2#9752 AS county#10072, Province_State#9753, Country_Region#9754, Confirmed#9758L, Deaths#9759L, Recovered#9760L, Active#9761L, Incident_Ratet#9763, Case_Fatality_Ratio#9764, date#9878]
 +- Project [FIPS#9751, Admin2#9752, Province_State#9753, Country_Region#9754, Confirmed#9758L, Deaths#9759L, Recovered#9760L, Active#9761L, Incident_Ratet#9763, Case_Fatality_Ratio#9764, date#9878]
 +- Filter AtLeastNNulls(n, Confirmed#9758L,Deaths#9759L,Recovered#9760L,Active#9761L,date#9878)
 +- Project [FIPS#9751, Admin2#9752, Province_State#9753, Country_Region#9754, Last_Update#9755, Lat#9756, Long_#9757, Confirmed#9758L, Deaths#9759L, Recovered#9760L, Active#9761L, CombinedKey#9762, Incident_Ratet#9763, Case_Fatality_Ratio#9764, to_date('Last_Update, None) AS date#9878]
 +- Relation[FIPS#9751,Admin2#9752,Province_State#9753,Country_Region#9754,Last_Update#9755,Lat#9756,Long_#9757,Confirmed#9758L,Deaths#9759L,Recovered#9760L,Active#9761L,CombinedKey#9762,Incident_Ratet#9763,Case_Fatality_Ratio#9764] csv

== Optimized Logical Plan ==
Project [Country_Region#9754, date#9878, Confirmed#9758L, Active#9761L]
+- Sort [date#9878 DESC NULLS LAST], true
 +- Project [Country_Region#9754, Confirmed#9758L, Active#9761L, date#9878]
 +- Filter (((isnotnull(Country_Region#9754) AND isnotnull(date#9878)) AND (Country_Region#9754 = Poland)) AND (year(date#9878) = 2020))
 +- InMemoryRelation [FIPS#9751, county#10072, Province_State#9753, Country_Region#9754, Confirmed#9758L, Deaths#9759L, Recovered#9760L, Active#9761L, Incident_Ratet#9763, Case_Fatality_Ratio#9764, date#9878], StorageLevel(disk, memory, deserialized, 1 replicas)
 +- *(1) Project [FIPS#9751, Admin2#9752 AS county#10072, Province_State#9753, Country_Region#9754, Confirmed#9758L, Deaths#9759L, Recovered#9760L, Active#9761L, Incident_Ratet#9763, Case_Fatality_Ratio#9764, cast(

In [0]:
global_data.select("Country_Region", "Confirmed", "Active", "date").filter((col("Country_Region") == "Poland") & (col("Active") >=  300000)).show()

+--------------+---------+------+----------+
Country_Region|Confirmed|Active| date|
+--------------+---------+------+----------+
 Poland| 1067870|325243|2020-12-08|
 Poland| 1063449|336640|2020-12-07|
 Poland| 1076180|316743|2020-12-09|
 Poland| 1054273|346502|2020-12-06|
 Poland| 1041846|356074|2020-12-05|
 Poland| 1028610|363222|2020-12-04|
 Poland| 1013747|372559|2020-12-03|
 Poland| 985075|408617|2020-11-30|
 Poland| 999924|384736|2020-12-02|
 Poland| 990811|396147|2020-12-01|
 Poland| 973593|418574|2020-11-29|
 Poland| 958416|425633|2020-11-28|
 Poland| 941112|430675|2020-11-27|
 Poland| 924422|439907|2020-11-26|
 Poland| 909066|440035|2020-11-25|
 Poland| 876333|423691|2020-11-24|
 Poland| 843475|423112|2020-11-22|
 Poland| 861331|423742|2020-11-23|
 Poland| 819262|424748|2020-11-21|
 Poland| 796798|422824|2020-11-20|
+--------------+---------+------+----------+
only showing top 20 rows

##### Get days with the most active cases in Poland

In [0]:
global_data.filter((col("Country_Region") == "Poland")).select("date","Active").orderBy(col("Active").desc()).limit(5).show()

+----------+------+
 date|Active|
+----------+------+
2020-11-25|440035|
2020-11-26|439907|
2020-11-27|430675|
2020-11-28|425633|
2020-11-21|424748|
+----------+------+

#### Perfrom some aggregations

##### Common Aggragation Functions
* count(col)
* countDistinct(col)
* min(col)
* max(col)
* sum(col)
* sumDistinct(col)
* avg(col)

In [0]:
global_data.select(countDistinct("Country_Region")).show()

+------------------------------+
count(DISTINCT Country_Region)|
+------------------------------+
 193|
+------------------------------+

In [0]:
global_data.select(max(col("Confirmed"))).show()

+--------------+
max(Confirmed)|
+--------------+
 3322195|
+--------------+

In [0]:
#global_data.groupBy("Country_Region").show()
global_data.groupBy("Country_Region").agg(count("*")).show()

+--------------+--------+
Country_Region|count(1)|
+--------------+--------+
 Chad| 308|
 Paraguay| 308|
 Russia| 20315|
 Yemen| 296|
 Senegal| 308|
 Cabo Verde| 308|
 Sweden| 5108|
 Guyana| 308|
 Burma| 304|
 Eritrea| 308|
 Philippines| 308|
 Djibouti| 308|
 Malaysia| 308|
 Singapore| 308|
 Fiji| 308|
 Turkey| 308|
 Malawi| 301|
 Iraq| 308|
 Germany| 4499|
 Comoros| 276|
+--------------+--------+
only showing top 20 rows

In [0]:
num_of_provinces = global_data.groupBy("Country_Region").agg(countDistinct("Province_State"))
num_of_provinces.show()
num_of_provinces.where(col("Country_Region") == "Poland").show()

+--------------+---------------------+
Country_Region|count(Province_State)|
+--------------+---------------------+
 Chad| 0|
 Paraguay| 0|
 Russia| 83|
 Yemen| 0|
 Senegal| 0|
 Sweden| 21|
 Cabo Verde| 0|
 Guyana| 0|
 Burma| 0|
 Eritrea| 0|
 Philippines| 0|
 Djibouti| 0|
 Malaysia| 0|
 Singapore| 0|
 Fiji| 0|
 Turkey| 0|
 Malawi| 0|
 Iraq| 0|
 Germany| 17|
 Comoros| 0|
+--------------+---------------------+
only showing top 20 rows

+--------------+---------------------+
Country_Region|count(Province_State)|
+--------------+---------------------+
 Poland| 0|
+--------------+---------------------+

##### Get cases for each Country

In [0]:
last_data = global_data.where(col("date") == "2021-01-31")

In [0]:
global_data_country = last_data.groupBy("Country_Region").agg(sum(col("Confirmed")).alias("Confirmed"), 
                                        sum(col("Active")).alias("Active"), 
                                        sum(col("Deaths")).alias("Deaths"), 
               )
gobal_data_country.orderBy(col("Active").desc()).show()

+--------------+---------+--------+------+
Country_Region|Confirmed| Active|Deaths|
+--------------+---------+--------+------+
 US| 26074354|25635589|438765|
United Kingdom| 3806993| 3797217|105777|
 France| 3236685| 2930353| 76006|
 Spain| 2743119| 2534424| 58319|
 Mexico| 1857230| 1699156|158074|
 Peru| 1133022| 1092165| 40857|
 Netherlands| 988154| 961429| 14066|
 Brazil| 9176975| 804359|223945|
 Belgium| 695277| 695277| 0|
 Sweden| 566957| 555366| 11591|
 Russia| 3790265| 473853| 71556|
 Italy| 2541783| 463352| 88279|
 Serbia| 393897| 389897| 4000|
 Germany| 2224911| 228877| 57105|
 Poland| 1508674| 207018| 37082|
 Switzerland| 521320| 194350| 9370|
 Ukraine| 1258093| 184558| 23769|
 Portugal| 711018| 179939| 12179|
 Indonesia| 1066313| 174083| 29728|
 India| 10746174| 168784|154274|
+--------------+---------+--------+------+
only showing top 20 rows

In [0]:
global_data_country.orderBy(col("Confirmed").desc()).show()

+--------------+---------+--------+------+
Country_Region|Confirmed| Active|Deaths|
+--------------+---------+--------+------+
 US| 26074354|25635589|438765|
 India| 10746174| 168784|154274|
 Brazil| 9176975| 804359|223945|
United Kingdom| 3806993| 3797217|105777|
 Russia| 3790265| 473853| 71556|
 France| 3236685| 2930353| 76006|
 Spain| 2743119| 2534424| 58319|
 Italy| 2541783| 463352| 88279|
 Turkey| 2470901| 89627| 25865|
 Germany| 2224911| 228877| 57105|
 Colombia| 2086806| 108755| 53650|
 Argentina| 1922264| 161465| 47931|
 Mexico| 1857230| 1699156|158074|
 Poland| 1508674| 207018| 37082|
 South Africa| 1449236| 112364| 43951|
 Iran| 1411731| 150949| 57889|
 Ukraine| 1258093| 184558| 23769|
 Peru| 1133022| 1092165| 40857|
 Indonesia| 1066313| 174083| 29728|
 Netherlands| 988154| 961429| 14066|
+--------------+---------+--------+------+
only showing top 20 rows

In [0]:
global_data_country = global_data.groupBy("Country_Region", "date").agg(sum(col("Confirmed")).alias("Confirmed"), 
                                        sum(col("Active")).alias("Active"), sum(col("Deaths")).alias("Deaths"))

In [0]:
global_data_country.show()

+-------------------+----------+---------+------+------+
 Country_Region| date|Confirmed|Active|Deaths|
+-------------------+----------+---------+------+------+
 Luxembourg|2021-01-29| 50228| 2148| 574|
Antigua and Barbuda|2021-01-30| 218| 37| 6|
 Equatorial Guinea|2021-01-25| 5401| 118| 86|
 Vanuatu|2021-01-25| 1| 0| 0|
 Congo (Kinshasa)|2021-01-27| 22048| 6419| 661|
 Paraguay|2021-01-27| 129394| 22173| 2651|
 Saudi Arabia|2021-01-24| 366185| 2107| 6350|
 Singapore|2021-01-24| 59260| 216| 29|
 Guinea|2021-01-28| 14435| 407| 82|
 Cambodia|2021-01-22| 456| 57| 0|
 Uzbekistan|2021-01-22| 78219| 944| 620|
 Ethiopia|2021-01-18| 131195| 13018| 2030|
 Hungary|2021-01-19| 352703|110853| 11409|
 Maldives|2021-01-20| 14633| 969| 49|
 Sri Lanka|2021-01-13| 49537| 6672| 244|
 Vietnam|2021-01-13| 1520| 124| 35|
 Zambia|2021-01-13| 29757| 8188| 495|
 Uganda|2021-01-12| 37808| 24564| 302|
 Sweden|2021-01-11| 489471|480038| 9433|
 Nicaragua|2021-01-08| 6097| 1706| 166|
+-------------------+----------+---------+------+------+
only showing top 20 rows

In [0]:
global_data_country.groupBy("Country_Region").agg(max(col("Confirmed")), max(col("Active"))).show()

+--------------+--------------+-----------+
Country_Region|max(Confirmed)|max(Active)|
+--------------+--------------+-----------+
 Chad| 3347| 829|
 Paraguay| 132548| 26225|
 Russia| 3790265| 558147|
 Yemen| 2120| 580|
 Senegal| 26213| 4419|
 Sweden| 566957| 555366|
 Cabo Verde| 13981| 1140|
 Guyana| 7581| 1146|
 Philippines| 523516| 105008|
 Burma| 139864| 19859|
 Eritrea| 2135| 798|
 Djibouti| 5931| 2390|
 Singapore| 59507| 20799|
 Malaysia| 209661| 47388|
 Fiji| 55| 17|
 Turkey| 2470901| 1306587|
 Malawi| 23497| 14562|
 Iraq| 618922| 63157|
 Germany| 2224911| 380989|
 Comoros| 2718| 975|
+--------------+--------------+-----------+
only showing top 20 rows

In [0]:
global_data_country.groupBy("Country_Region").agg(avg(col("Active"))).show()

+--------------+-----------+
Country_Region|avg(Active)|
+--------------+-----------+
 Chad| 787.0|
 Paraguay| 22210.0|
 Russia| 473853.0|
 Yemen| 79.0|
 Senegal| 3834.0|
 Cabo Verde| 745.0|
 Sweden| 555366.0|
 Guyana| 835.0|
 Burma| 12298.0|
 Eritrea| 534.0|
 Philippines| 36943.0|
 Djibouti| 24.0|
 Malaysia| 47388.0|
 Singapore| 282.0|
 Fiji| 0.0|
 Turkey| 89627.0|
 Malawi| 14492.0|
 Iraq| 15024.0|
 Germany| 228877.0|
 Comoros| 950.0|
+--------------+-----------+
only showing top 20 rows

## Using SQL in Spark SQL

In [0]:
infoDF = spark.sql("select current_date() as today , 1 + 100 as value")
infoDF.show()


+----------+-----+
 today|value|
+----------+-----+
2021-01-26| 101|
+----------+-----+

In [0]:
global_data.createOrReplaceTempView("covid")

In [0]:
spark.sql("SELECT * FROM covid").show()

+----+------+--------------------+-------------------+---------+------+---------+------+------------------+-------------------+----------+
FIPS|county| Province_State| Country_Region|Confirmed|Deaths|Recovered|Active| Incident_Ratet|Case_Fatality_Ratio| date|
+----+------+--------------------+-------------------+---------+------+---------+------+------------------+-------------------+----------+
null| null| null| Afghanistan| 55008| 2400| 47609| 4999| 141.3057905550098| 4.363001745200698|2021-01-31|
null| null| null| Albania| 77251| 1369| 46927| 28955| 2684.376954618111| 1.7721453443968362|2021-01-31|
null| null| null| Algeria| 107122| 2888| 73152| 31082| 244.2860937195952| 2.695991486342675|2021-01-31|
null| null| null| Andorra| 9885| 101| 9045| 739|12793.632304406909| 1.0217501264542235|2021-01-31|
null| null| null| Angola| 19782| 464| 18015| 1303| 60.18937105971387| 2.3455666767768677|2021-01-31|
null| null| null|Antigua and Barbuda| 218| 7| 175| 36|222.61253165591046| 3.2110091743119265|2021-01-31|
null| null| null| Argentina| 1922264| 47931| 1712868|161465| 4253.19383268928| 2.493466037963568|2021-01-31|
null| null| null| Armenia| 166901| 3071| 157314| 6516| 5632.393526802136| 1.8400129418038238|2021-01-31|
null| null|Australian Capita...| Australia| 118| 3| 115| 0|27.563653352020555| 2.542372881355932|2021-01-31|
null| null| New South Wales| Australia| 5104| 54| 3255| 1795| 62.87262872628726| 1.0579937304075235|2021-01-31|
null| null| Northern Territory| Australia| 98| 0| 93| 5| 39.90228013029316| 0.0|2021-01-31|
null| null| Queensland| Australia| 1309| 6| 1295| 8| 25.58889649105659|0.45836516424751717|2021-01-31|
null| null| South Australia| Australia| 597| 4| 592| 1|33.988044406490175| 0.6700167504187605|2021-01-31|
null| null| Tasmania| Australia| 234| 13| 221| 0| 43.69747899159664| 5.555555555555555|2021-01-31|
null| null| Victoria| Australia| 20448| 820| 19608| 20|308.42094149233014| 4.010172143974961|2021-01-31|
null| null| Western Australia| Australia| 903| 9| 878| 16| 34.32676955827568| 0.9966777408637874|2021-01-31|
null| null| null| Austria| 413208| 7703| 391076| 14429| 4587.937466690353| 1.8641943040793014|2021-01-31|
null| null| null| Azerbaijan| 230066| 3126| 222960| 3980| 2269.080078014237| 1.358740535324646|2021-01-31|
null| null| null| Bahamas| 8174| 176| 6771| 1227| 2078.586540808853| 2.1531685833129437|2021-01-31|
null| null| null| Bahrain| 102626| 372| 98160| 4094| 6031.207410981421|0.36248124257010894|2021-01-31|
+----+------+--------------------+-------------------+---------+------+---------+------+------------------+-------------------+----------+
only showing top 20 rows

In [0]:
most_cases = spark.sql("SELECT max(Confirmed) AS max_cases, max(Deaths) AS max_deaths, Country_Region\
          FROM covid\
          GROUP BY Country_Region\
          ORDER BY max_cases DESC\
          LIMIT 10"
         )
most_cases.show()




+---------+----------+--------------+
max_cases|max_deaths|Country_Region|
+---------+----------+--------------+
 3322195| 92920|United Kingdom|
 3167274| 75478| France|
 2470901| 25865| Turkey|
 2023814| 51042| India|
 1922264| 47931| Argentina|
 1773024| 52954| Brazil|
 1508674| 37082| Poland|
 1449236| 43951| South Africa|
 1411731| 57889| Iran|
 1111391| 23689| US|
+---------+----------+--------------+

In [0]:
spark.sql("SELECT Country_Region, max(Confirmed) as maxConfirmed \
            FROM covid \
            GROUP BY Country_Region").agg(sum(col("maxConfirmed"))).show()



+-----------------+
sum(maxConfirmed)|
+-----------------+
 44605522|
+-----------------+

In [0]:
most_cases.write.format("json").mode("overwrite").save("/data/output/somejson")